In [68]:
import numpy as np
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model_iMM904 = read_sbml_model('iMM904.xml')
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite
new_reaction1 = Reaction('phy_syn') #PHYtoene SYNtase
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite

new_reaction1.add_metabolites({model_iMM904.metabolites.ggdp_c: -2,
                              phy: 1, # our newly created metabolites
                               model_iMM904.metabolites.ppi_c: 2
                                })
model_iMM904.add_reactions([new_reaction1])
new_reaction2 = Reaction('neu_syn') #NEUrosporene SYNtase
#Adding the metabolite
neu = Metabolite(id='neu', name='Neurosporene', compartment='c') 
#AH2 = Metabolite(id='AH2', name='Reduced acceptor', compartment='c') 
#A = Metabolite(id='A', name='Acceptor', compartment='c') 
new_reaction2.add_metabolites({
    model_iMM904.metabolites.nadp_c: -3,
    phy: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: 3,
    neu: 1,
    })
model_iMM904.add_reactions([new_reaction2])
new_reaction3 = Reaction('lyc_syn') #LYCopene SYNtase
lyc = Metabolite(id='lyc', name='Lycopene', compartment='c') #Adding the metabolite
new_reaction3.add_metabolites({
    neu: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: -1, 
    model_iMM904.metabolites.o2_c: -1,
    lyc: 1,
    model_iMM904.metabolites.nadp_c: 1,
    model_iMM904.metabolites.h2o_c: 2,
    })
model_iMM904.add_reactions([new_reaction3])
new_reaction4 = Reaction('beta_syn') #BETA-carotene SYNtase
beta = Metabolite(id='beta', name='Beta-carotene', compartment='c') #Adding the metabolite
new_reaction4.add_metabolites({
    lyc: -1, # our newly created metabolites
    beta: 1,
    })
model_iMM904.add_reactions([new_reaction4])
new_reaction5 = Reaction('EX_beta') #BETA-carotene SYNtase
new_reaction5.add_metabolites({
    beta: -1 #Excretion of beta-carotene
    })
model_iMM904.add_reactions([new_reaction5])
with model_iMM904:
    model_iMM904.objective = model_iMM904.reactions.EX_beta 
    print("Maximum beta-carotene flow: ",model_iMM904.optimize().objective_value)

Maximum beta-carotene flow:  0.35143769968051247


In [27]:
carbon_source_exchanges=[]
for exchange in model_iMM904.exchanges:
    for metabolite in exchange.metabolites:
        if "C" in metabolite.elements:
            carbon_source_exchanges.append(exchange.id)

['EX_epistest_SC_e', 'EX_epist_e', 'EX_ergst_e', 'EX_ergstest_SC_e', 'EX_13BDglcn_e', 'EX_etha_e', 'EX_2hb_e', 'EX_etoh_e', 'EX_fecost_e', 'EX_2mbac_e', 'EX_2mbald_e', 'EX_2mbtoh_e', 'EX_2mppal_e', 'EX_2phetoh_e', 'EX_fecostest_SC_e', 'EX_fmn_e', 'EX_3c3hmp_e', 'EX_3mbald_e', 'EX_for_e', 'EX_fru_e', 'EX_3mop_e', 'EX_4abut_e', 'EX_fum_e', 'EX_g3pc_e', 'EX_4abz_e', 'EX_5aop_e', 'EX_g3pi_e', 'EX_gal_e', 'EX_8aonn_e', 'EX_Nbfortyr_e', 'EX_abt_e', 'EX_ac_e', 'EX_acald_e', 'EX_aces_e', 'EX_galur_e', 'EX_gam6p_e', 'EX_gcald_e', 'EX_glc__D_e', 'EX_gln__L_e', 'EX_glu__L_e', 'EX_glx_e', 'EX_ade_e', 'EX_adn_e', 'EX_akg_e', 'EX_ala__L_e', 'EX_gly_e', 'EX_glyc_e', 'EX_gsn_e', 'EX_gthox_e', 'EX_gthrd_e', 'EX_alltn_e', 'EX_alltt_e', 'EX_amet_e', 'EX_arab__D_e', 'EX_arab__L_e', 'EX_gua_e', 'EX_arg__L_e', 'EX_hdca_e', 'EX_hdcea_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_btd_RR_e', 'EX_btn_e', 'EX_hexc_e', 'EX_his__L_e', 'EX_hxan_e', 'EX_iamac_e', 'EX_iamoh_e', 'EX_ibutac_e', 'EX_chol_e', 'EX_cit_e', 'EX_co2

In [69]:
#First we sort for all the exchange reactions that 
carbon_source_exchanges=[] #creates empty list to save carbon sources
for exchange in model_iMM904.exchanges: #goes through all exchange reactions
    for metabolite in exchange.metabolites: #goes trhough the metabolites in all the reactions
        if "C" in metabolite.elements: #Checks if it contains carbon
            carbon_source_exchanges.append(exchange.id) #saves in case this is true
            
#First find the initial max growth rate and max flux so we can compare later
#These are all for the standard medium settings with glucose
with model_iMM904: #We do this under model to avoid our objective chaning futher in the notebook 
    initial_growth = model_iMM904.slim_optimize() #Find max growth
    model_iMM904.objective = model_iMM904.reactions.EX_beta #sets objective to the beta-carotene flow
    initial_beta_flow = model_iMM904.optimize().objective_value #Find max beta-carotene flow
#prints the parameters with two decimals
print("Max growth rate on glucose:",round(initial_growth,2))
print("Max beta-carotene flow on glucose:",round(initial_beta_flow,2))
print(" ")
    
potential_better_Csources = [] #makes emtpy list so we can save potential better C-sources for further analysis
for i in range (0,len(carbon_source_exchanges)): #we go through all carbon sources on the previous made list
    with model_iMM904: #We do this under model to avoid our objective chaning futher in the loop and notebook
        medium=model_iMM904.medium #resets medium to standard
        medium['EX_glc__D_e']=0 #sets glucose level to 0
        medium[carbon_source_exchanges[i]]=10 #sets level of carbon source to 10
        model_iMM904.medium =medium #defines medium model
        solution_growth = model_iMM904.slim_optimize(error_value=0.) #save max growth rate
        if solution_growth >= initial_growth: #checks if max growth rate is equal or higher than the standard
            model_iMM904.objective = model_iMM904.reactions.EX_beta #sets objective to the beta-carotene flow
            solution_beta = model_iMM904.optimize().objective_value #save max beta-carotene flow
            if solution_beta >= initial_beta_flow: #checks if max beta-carotene flow is equal or higher than the standard
                potential_better_Csources.append(carbon_source_exchanges[i]) #In case both are true saves the carbon source in the list
                name=model_iMM904.reactions.get_by_id(carbon_source_exchanges[i]) #gets the reaction name for the carbon source
                print(name.name,":") #print name 
                print("Maximum growth rate for this crabonsource: ",round(solution_growth,2)) #prints rounded max growth rate
                print("Maximum beta-carotene flow for this carbon-source:: ",round(model_iMM904.optimize().objective_value,2)) #prints max beta-carotene flow
                print(" ")#just spacing 

Max growth rate on glucose: 0.29
Max beta-carotene flow on glucose: 0.35
 
D-Glucosamine 6-phosphate exchange :
Maximum growth rate for this crabonsource:  0.41
Maximum beta-carotene flow for this carbon-source::  0.45
 
Adenosine exchange :
Maximum growth rate for this crabonsource:  0.37
Maximum beta-carotene flow for this carbon-source::  0.41
 
Guanosine exchange :
Maximum growth rate for this crabonsource:  0.37
Maximum beta-carotene flow for this carbon-source::  0.41
 
S Adenosyl L methionine exchange :
Maximum growth rate for this crabonsource:  0.37
Maximum beta-carotene flow for this carbon-source::  0.41
 
Inosine exchange :
Maximum growth rate for this crabonsource:  0.37
Maximum beta-carotene flow for this carbon-source::  0.41
 
Maltose exchange :
Maximum growth rate for this crabonsource:  0.52
Maximum beta-carotene flow for this carbon-source::  0.61
 
Melibiose exchange :
Maximum growth rate for this crabonsource:  0.52
Maximum beta-carotene flow for this carbon-source

In [26]:
model_iMM904.slim_optimize()

0.28786570370404924

In [59]:
model_iMM904.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

In [64]:
with model_iMM904:
    medium=model_iMM904.medium
    medium['EX_o2_e']=0
    model_iMM904.medium =medium
    print(model_iMM904.medium)
    print(model_iMM904.slim_optimize())

{'EX_fe2_e': 999999.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_pi_e': 999999.0}
4.2156280169557445e-31
